In [2]:
source("utils/plot.R")

In [5]:
signals <- read_tsv(snakemake@input[[1]], col_types=cols()) %>%
    mutate(
        feature=feat_factor(feature),
        position = position - 5,
        PositionInKbp = position*(window_size/2)/1000,
        WindowSize=paste(round(window_size/1000, 1), 'kb', sep=''),
        WindowSize=factor(WindowSize, levels=str_sort(unique(WindowSize), numeric=TRUE)),
        dataset=str_replace_all(dataset, c('sweep'='STC1'))
    )

In [7]:
# Pick K window sizes to use

K <- 5
acceptable_ws <- quantile(signals$window_size, probs=c(0, 0.25, 0.5, 0.75, 1))
signals <- signals %>%
    filter(window_size %in% acceptable_ws)

In [10]:
signals_fig <- ggplot(signals) +
    geom_line(aes(x=PositionInKbp, y=value, colour=feature)) +
    facet_grid(rows=vars(dataset), cols=vars(WindowSize), switch='y', scales='free_x') +
    labs(
        x='Position (kb)',
        y='Normalized statistic'
    ) +
    turkana_colour +
    turkana_theme +
    theme(
        panel.grid.major.y = element_blank(),
        axis.text.y = element_blank(),
        axis.title.y = element_blank(),
        strip.text.y.left = element_text(angle = 0),
        panel.border = element_rect(colour='grey', fill=NA),
        legend.title = element_blank(),
        axis.text.x = element_text(angle=45, hjust=1),
        panel.spacing.x = unit(0.4, "lines"),
    )

In [12]:
turkana_save(snakemake@output[[1]], signals_fig, width=6)